In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
che_df = pd.read_csv('CHE_dataset_cleaned.csv')

che_df.head()

,AREA,INT_SQFT,N_BEDROOM,N_BATHROOM,SALES_PRICE
0,Karapakkam,1004.0,1.0,1.0,76.00
1,Anna Nagar,1986.0,2.0,1.0,217.18
2,Adyar,909.0,1.0,1.0,131.59
3,Velachery,1855.0,3.0,2.0,96.30
4,Karapakkam,1226.0,1.0,1.0,74.06


In [3]:
che_df['price_per_sqft']  = (che_df['SALES_PRICE']*100000) / che_df['INT_SQFT']

che_df.head()

,AREA,INT_SQFT,N_BEDROOM,N_BATHROOM,SALES_PRICE,price_per_sqft
0,Karapakkam,1004.0,1.0,1.0,76.00,7569.721116
1,Anna Nagar,1986.0,2.0,1.0,217.18,10935.548842
2,Adyar,909.0,1.0,1.0,131.59,14476.347635
3,Velachery,1855.0,3.0,2.0,96.30,5191.374663
4,Karapakkam,1226.0,1.0,1.0,74.06,6040.783034


In [4]:
#che_df['AREA'].apply(lambda x: x.strip())

loc_stats = che_df.groupby('AREA')['AREA'].agg('count').sort_values(ascending = False)
loc_stats[loc_stats<10]

AREA
Chrompt      9
Chrmpet      6
Chormpet     6
TNagar       5
Karapakam    3
Ana Nagar    3
Ann Nagar    2
Velchery     2
Adyr         1
KKNagar      1
Name: AREA, dtype: int64

In [5]:
che_df['AREA'] = che_df['AREA'].apply(lambda x: 'Others' if x in loc_stats[loc_stats<10] else x)

In [6]:
che_df['AREA'].value_counts()

AREA
Chrompet      1681
Karapakkam    1363
KK Nagar       996
Velachery      979
Anna Nagar     782
Adyar          773
T Nagar        496
Others          38
Name: count, dtype: int64

In [7]:
che_df.head()

,AREA,INT_SQFT,N_BEDROOM,N_BATHROOM,SALES_PRICE,price_per_sqft
0,Karapakkam,1004.0,1.0,1.0,76.00,7569.721116
1,Anna Nagar,1986.0,2.0,1.0,217.18,10935.548842
2,Adyar,909.0,1.0,1.0,131.59,14476.347635
3,Velachery,1855.0,3.0,2.0,96.30,5191.374663
4,Karapakkam,1226.0,1.0,1.0,74.06,6040.783034


In [8]:
che_df.shape

(7108, 6)

In [9]:
che_df = che_df[~(che_df['INT_SQFT']/che_df['N_BEDROOM']<300)]

In [10]:
che_df.shape

(7108, 6)

In [11]:
def remove_outlier(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('AREA'):
        mean = np.mean(subdf.price_per_sqft)
        std = np.std(subdf.price_per_sqft)
        reduced_df = subdf[(subdf.price_per_sqft > (mean - std)) & (subdf.price_per_sqft <= (mean + std))]
        df_out = pd.concat([df_out, reduced_df], ignore_index = True)
    return df_out

In [12]:
che_df = remove_outlier(che_df)

In [13]:
che_df.shape

(4566, 6)

In [14]:
che_df

,AREA,INT_SQFT,N_BEDROOM,N_BATHROOM,SALES_PRICE,price_per_sqft
0,Adyar,1056.0,1.0,1.0,93.86,8888.257576
1,Adyar,1130.0,1.0,1.0,133.83,11843.362832
2,Adyar,730.0,1.0,1.0,88.48,12120.547945
3,Adyar,868.0,1.0,1.0,90.42,10417.050691
4,Adyar,718.0,1.0,1.0,55.70,7757.660167
...,...,...,...,...,...,...
4561,Velachery,1907.0,3.0,2.0,140.25,7354.483482
4562,Velachery,1941.0,3.0,2.0,94.20,4853.168470
4563,Velachery,1897.0,3.0,2.0,108.18,5702.688455
4564,Velachery,1614.0,2.0,1.0,83.51,5174.101611


In [15]:
che_df.to_csv('CHE_feature_eng.csv')